## Description
This code prices options (European/American calls/puts) on a simple stock modelled using the binomial tree model, according to the no arbitrage principle.

Universe parameters:
- Risk-free rate (r) (constant for simplicity, but could make a better, time dependent model).

Stock parameters:
- Initial value (S)
- Volatility (sigma).

Binomial tree parameters:
- Time step (dt)
- Time period of interest (T).

Option parameters:
- Exercise style (style)
- Option type (type)
- Strike (K)
- Maturity (equal to time period of interest (T), so not customizable beyond that parameter).

In [ ]:
import math
import numpy as np
np.set_printoptions(suppress=True)

'''
This code prices options (European/American calls/puts) on a simple stock modelled using binomial tree model, according to the no arbitrage principle.
    Universe parameters: Risk-free rate (r) (constant for simplicity, but could make a better, time dependent model).
    Stock parameters: Initial value (S), volatility (sigma).
    Binomial tree parameters: Time step (dt), time period of interest (T).
    Option parameters: Exercise style (style), option type (type), strike (K), maturity (equal to time period of interest (T), so not customizable beyond that parameter).
'''

S = 100 # Initial value of the stock, in $
sigma = 0.06 # Volatility, in months^(-1/2)
r = 0.0025 # Risk-free rate, in months^(-1)

def create_stock_tree(T, dt):
    '''
    Creates a stock binomial tree.

    Parameters:
        T (float): Time to maturity (or time period for which we want to model the stock), measured in months.
        dt (float): Time step, measured in months.

    Returns:
        tuple (np.ndarray, float): Tuple containing binomial tree modelling the evolution of the stock, and risk-neutral probability of upward movement.
    '''
    N = round(T / dt) # Total number of time steps
    u = math.exp(sigma * math.sqrt(dt)) # Up factor
    d = math.exp(-sigma * math.sqrt(dt)) # Down factor
    rn_p = (math.exp(r * dt) - d) / (u - d) # Risk-neutral probability of upward movement

    tree = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(i + 1):
            tree[j, i] = round(S * u ** (i - j) * d ** j, 2)
    
    return (tree, rn_p)


def create_option_tree(style, type, K, tree_p):
    '''
    Creates a binomial tree containing the value of the option in each node, matches the shape of the stock tree

    Parameters:
        style (string, "E" or "A"): Exercise style, is the option European or American?
        type (string, "C" or "P"): Option type, is the option a call or a put?
        K (float): Strike price.
        tree_p (tuple (np.ndarray, float)): Tuple containing binomial tree modelling the evolution of the stock until maturity, and risk-neutral probability of upward movement.

    Returns:
        (np.ndarray): Binomial tree modelling the evolution of the option value, measured in present value $ (and, most importantly, obtaining its value at t = 0).
    '''
    tree_s = tree_p[0] # Stock binomial tree
    rn_p = tree_p[1] # Risk-neutral probability of upward movement
    N = np.shape(tree_s)[0] - 1 # Number of time steps

    tree = np.zeros((N + 1, N + 1)) # Initialize option value tree
    # Fill up the tree, by computing the known possible values at maturity and performing backwards induction
    if style == "E": # For Europeans

        if type == "C": # For calls
            for j in range(N + 1):
                tree[j, N] = max((tree_s[j, N] - K) * math.exp(-r * N), 0)
        elif type == "P": # For puts
            for j in range(N + 1):
                tree[j, N] = max((K - tree_s[j, N]) * math.exp(-r * N), 0)
        else:
            raise ValueError(f'Please introduce a valid type: either "C" (Call) or "P" (Put)')
        
        for i in range(N, 0, -1):
                for j in range(i):
                    tree[j, i - 1] = rn_p * tree[j, i] + (1 - rn_p) * tree[j + 1, i]
        
    elif style == "A": # For Americans

        if type == "C": # For calls
            for j in range(N + 1):
                tree[j, N] = max((tree_s[j, N] - K) * math.exp(-r * N), 0)
            
            for i in range(N, 0, -1):
                for j in range(i):
                    tree[j, i - 1] = max(rn_p * tree[j, i] + (1 - rn_p) * tree[j + 1, i], (tree_s[j, i - 1] - K) * math.exp(-r * (i - 1)))

        elif type == "P": # For puts
            for j in range(N + 1):
                tree[j, N] = max((K - tree_s[j, N]) * math.exp(-r * N), 0)

            for i in range(N, 0, -1):
                for j in range(i):
                    tree[j, i - 1] = max(rn_p * tree[j, i] + (1 - rn_p) * tree[j + 1, i], (K - tree_s[j, i - 1]) * math.exp(-r * (i - 1)))

        else:
            raise ValueError(f'Please introduce a valid type: either "C" (Call) or "P" (Put)')
    else:
        raise ValueError(f'Please introduce a valid style: either "E" (European) or "A" (American)')
    
    tree = np.round(tree, 2)
    return tree

dt = 1 # Time step, in months
T = 6 # Time to maturity, in months
stock_tree = create_stock_tree(T, dt)

european_call = create_option_tree("E", "C", 100, stock_tree)
american_call = create_option_tree("A", "C", 100, stock_tree)
european_put = create_option_tree("E", "P", 100, stock_tree)
american_put = create_option_tree("A", "P", 100, stock_tree)

print(european_call)
print(american_call)
print(european_put)
print(american_put)

[[ 6.35  9.71 14.34 20.31 27.35 34.79 42.68]
 [ 0.    2.92  4.96  8.22 13.12 19.72 26.72]
 [ 0.    0.    0.82  1.63  3.21  6.35 12.56]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]]
[[ 6.35  9.71 14.34 20.31 27.35 34.79 42.68]
 [ 0.    2.92  4.96  8.22 13.12 19.72 26.72]
 [ 0.    0.    0.82  1.63  3.21  6.35 12.56]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.  ]]
[[ 4.86  2.3   0.66  0.    0.    0.    0.  ]
 [ 0.    7.49  3.97  1.34  0.    0.    0.  ]
 [ 0.    0.   11.09  6.67  2.72  0.    0.  ]
 [ 0.    0.    0.   15.61 10.7   5.51  0.  ]
 [ 0.    0.    0.    0.   20.63 16.02 11.14]
 [ 0.    0.    0.    0.    0.   25.35 21.02]
 [ 0.    0.    0.    0.    0.    0.   29.78]]
[[ 5.09  2.4   0.69  0.    0.    0.    0.  ]
 [ 0.  